### Son iguales los datasets de huggingface y los .conll?

In [25]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict, Dataset
from pysentimiento.lince.ner import load_datasets, load_conll

train_dataset, dev_dataset, test_dataset = load_datasets("es", preprocess=False)

lince_ner = DatasetDict(
    train=train_dataset,
    validation=dev_dataset,
    test=test_dataset,
)

def load_dataset_from_conll(path):
    data = load_conll(path)
    words = [[x[0] for x in sentence] for sentence in data]
    langs = [[x[1] for x in sentence] for sentence in data]
    if len(data[0][0]) == 2:
        ner = [[None] * len(x) for x in data]
    else:
        ner = [[x[2] for x in sentence] for sentence in data]

    ## Sanity Check
    for w, l in zip(words, ner):
        assert len(w) == len(l)
    
    return Dataset.from_dict({
        "words": words,
        "lang": langs,
        "ner": ner,
    })


orig = DatasetDict(
    train=load_dataset_from_conll("../../../data/lince/ner_spaeng/train.conll"),
    validation=load_dataset_from_conll("../../../data/lince/ner_spaeng/dev.conll"),
    test=load_dataset_from_conll("../../../data/lince/ner_spaeng/test.conll"),
) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-f749fd5ca349b1d8.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-875716555d36ff1f.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-1afcf30fe2e4ebdb.arrow


In [27]:
from tqdm.auto import tqdm

problematic = []
for split in ["train", "validation", "test"]:

    assert len(lince_ner[split]) == len(orig[split])
    for idx, (ex, orig_ex) in tqdm(enumerate(zip(lince_ner[split], orig[split])), total=len(lince_ner[split])):
        if not ex["words"] == orig_ex["words"]:
            problematic.append(idx)

print(f"{len(problematic)} problematic examples")

  0%|          | 0/33611 [00:00<?, ?it/s]

  0%|          | 0/10085 [00:00<?, ?it/s]

  0%|          | 0/23527 [00:00<?, ?it/s]

0 problematic examples


Todo bien jefe!